In [4]:
import os
import pandas as pd
import numpy as np
import  datetime 

directory = r'/Users/meiyewang/Desktop/code/data/windsolar/ACTUAL/2019'
df1=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df2=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df3=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df4=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
df5=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
for filename in os.listdir(directory):
    if filename.endswith(".csv") :
        df=pd.read_csv(os.path.join(directory, filename))
        df1=pd.concat([df1,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df2=pd.concat([df2,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df3=pd.concat([df3,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df4=pd.concat([df4,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        df5=pd.concat([df5,df[(df['TRADING_HUB']=='ZP26') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
    else:
        continue

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(8749, 3)
(8749, 3)
(8749, 3)
(8749, 3)
(8749, 3)


In [5]:
df1.head()

,INTERVALSTARTTIME_GMT,RENEWABLE_TYPE,MW
0,2019-04-18T08:00:00-00:00,Solar,-2.91828
1,2019-04-18T09:00:00-00:00,Solar,-2.98123
2,2019-04-18T10:00:00-00:00,Solar,-2.96208
3,2019-04-18T11:00:00-00:00,Solar,-2.95255
4,2019-04-18T12:00:00-00:00,Solar,-2.93420


In [6]:
Solar=df1
Solar['MW']=df1['MW']+df3['MW']+df5['MW']
Solar=Solar.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)

ValueError: cannot reindex from a duplicate axis

In [8]:

from datetimerange import DateTimeRange 
def GetSolarWindRawYearlyData():
    directory = r'/Users/meiyewang/Desktop/code/data/windsolar/ACTUAL/2019'
    df1=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df2=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df3=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df4=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    df5=pd.DataFrame(columns=["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW'])
    for filename in os.listdir(directory):
        if filename.endswith(".csv") :
            df=pd.read_csv(os.path.join(directory, filename))
            df=pd.read_csv(os.path.join(directory, filename))
            df1=pd.concat([df1,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df2=pd.concat([df2,df[(df['TRADING_HUB']=='NP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df3=pd.concat([df3,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df4=pd.concat([df4,df[(df['TRADING_HUB']=='SP15') & (df['RENEWABLE_TYPE']=='Wind')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
            df5=pd.concat([df5,df[(df['TRADING_HUB']=='ZP26') & (df['RENEWABLE_TYPE']=='Solar')][["INTERVALSTARTTIME_GMT","RENEWABLE_TYPE",'MW']]])
        else:
            continue
    return df1,df2,df3,df4,df5

def SortTimeOrder(df):
    timelist=[datetime.datetime.strptime(df.iloc[i,0][:16], "%Y-%m-%dT%H:%M") for i in range(df.shape[0])]
    df["INTERVALSTARTTIME_GMT"]=timelist
    df_sort=df.sort_values(by="INTERVALSTARTTIME_GMT")
    df_sort.index=df_sort["INTERVALSTARTTIME_GMT"]
    return df_sort


ValueError: Invalid fill method. Expecting pad (ffill), backfill (bfill) or nearest. Got 99999

In [25]:

def FillMissingData(df_sort):
    
    time_range = DateTimeRange("2019-01-01 08:00:00", "2020-01-01 07:00:00")
    completeIndex= time_range.range(datetime.timedelta(hours=1))
    
    df_sort_complete=df_sort.reindex(completeIndex)
    df_sort_complete=df_sort_complete.interpolate(method='linear')
    return df_sort_complete


df1,df2,df3,df4,df5=GetSolarWindRawYearlyData()

df1_complete=FillMissingData(SortTimeOrder(df1))
df2_complete=FillMissingData(SortTimeOrder(df2))
df3_complete=FillMissingData(SortTimeOrder(df3))
df4_complete=FillMissingData(SortTimeOrder(df4))
df5_complete=FillMissingData(SortTimeOrder(df5))


In [27]:

def CompileWindSolar(df1_complete,df2_complete,df3_complete,df4_complete,df5_complete):
    Solar=df1_complete
    Solar['MW']=df1_complete['MW']+df3_complete['MW']+df3_complete['MW']
    Solar.index=range(8760)
    Solar=Solar.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)

    Wind=df2_complete
    Wind['MW']=df2_complete['MW']+df4_complete['MW']
    Wind.index=range(8760)
    Wind=Wind.drop(['INTERVALSTARTTIME_GMT','RENEWABLE_TYPE'],axis=1)
    return Solar,Wind



Solar,Wind=CompileWindSolar(df1_complete,df2_complete,df3_complete,df4_complete,df5_complete)



In [29]:
Solar[pd.isna(Solar['MW'])]

,MW
